### Step 1

Initialize the notebook, loading the configuration and importing libraries.

In [8]:
# Import the libraries required for the notebook
import asyncio

# Instatiate the config class
from config.notebook_config import *
config = notebook_config()
# Load config from file
config.load_config_from_file()


### Step 2

Load the data from a Word file, and store it in a data dictionary.

Replace filename by the fullpath to the Word file.

In [9]:
# Open the Word document
from docx import Document
filename = "example.docx"
document = Document(filename)

# Initialize an empty list for sessions and a None for the current session
sessions = []
current_session = None
current_session_content = ""

# Iterate through paragraphs and group content by "Heading 2"
for paragraph in document.paragraphs:
    if paragraph.style.style_id == "Heading2":
        # If there is a current session, append it to the sessions list
        if current_session is not None:
            current_session["content"] = current_session_content
            sessions.append(current_session)
        # Start a new session with the current paragraph as the title
        current_session = {"title": paragraph.text, "content": ""}
        current_session_content = ""
    else:
        # If there is a current session, append the paragraph to its content
        if current_session is not None:
            current_session_content += paragraph.text + "\n"

# Append the last session if it exists
if current_session is not None:
    current_session["content"] = current_session_content
    sessions.append(current_session)


print(f"Found {len(sessions)} sessions")
print(f"1st session: {sessions[0]['title']}")
print(f"1st content: {sessions[0]['content']}")


Found 2 sessions
1st session: Health data applied to rare diseases: review of the national plan for rare diseases 3 and prospects for the national plan for rare diseases 4 in 2024. Anne-Sophie LAPOINTE (France)
1st content: Context of rare diseases, obstacles: diagnostic error, limited information, geographic dispersion, access to treatment.
France had 3 rare disease plans so far. Third one being holistic, with care throughout the life span. 3rd plan is coming to the end now which had various ambitions: enable rapid diagnosis, innovate to treat, improving quality of life and autonomy of patients, structuring of date, (have a photo) etc… Guide, coordinate, share. 
France also has care guidelines for rett syndrome (only mentioned it).
All speech is more orientated for rare diseases in general rather than RS.
Spain has two ERNs: ERN EpiCARE and ERN Transplant??
BNDMR – the French national rare disease registry data (photo) www.bndmr.fr 
Plan 4: Focus on actions for neonatal screening, Inn

### Step 3

Initiate the Semantic Kernel and fire-up with OpenAI model.

In [10]:
# Load Semantic Kernel
import semantic_kernel
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

kernel = semantic_kernel.Kernel()
print("Kernel loaded.")

deployment = config.model
endpoint = config.endpoint
api_key = config.azure_api_key
kernel.add_chat_service("dv", AzureChatCompletion(deployment, endpoint, api_key))
print(f"Fire-up the kernel with {deployment}")

Kernel loaded.
Fire-up the kernel with gpt-35-turbo


### Step 4

Load plug-ins required to curate the congress notes.

In [11]:
plugins_directory = "../plugins-sk"

# Import the semantic functions
my_functions = kernel.import_semantic_skill_from_directory(plugins_directory, "CreateReport")

print("Plugins loaded.")

Plugins loaded.


### Step 5

Execute the "curation" process:
* First it will replace abbreviations and acronyms by their full meaning
* Then it will summarize the notes
* It will then extract the key concepts, in a report format
* Finally it will translate to Spanish

In [12]:
from semantic_kernel import ContextVariables

session_id = 1
input_str = sessions[session_id]["content"]
print(f"Input string: {input_str}")

# Replace abbreviations and acronyms from the input text
abbreviations_expansion = asyncio.create_task(kernel.run_async(my_functions["AbbreviationExpansion"], input_str=input_str))
abbreviations_expansion_result = await abbreviations_expansion
print("Abbreviations expansion result:")
print(abbreviations_expansion_result.result)

# Review notes from the previous step, and summarize the notes
note_review = asyncio.create_task(kernel.run_async(my_functions["NoteReview"], input_str=abbreviations_expansion_result.result))
note_review_result = await note_review

# Create the final session report, after the previous steps
report = asyncio.create_task(kernel.run_async(my_functions["Summarize"], input_str=abbreviations_expansion_result.result))
report_result = await report
print("Report result:")
print(report_result)

# Translate to Spanish
context = ContextVariables()
context["language"] = "spanish"
context["input"] = report_result.result
translate = asyncio.create_task(kernel.run_async(my_functions["TranslateNotes"], input_vars=context))
translate_result = await translate
print("Translate result:")
print(translate_result)

Input string: National diagnostics protocol (PNDS). Was created bases on the extensive literature review and with a group of experts. 
Role of general practitioner. They need to be the heart of the treatment plan. They need to be able to redirect for specialists that rs patients need depending on their phenotype. Which ones are really needed and which ones are optional. 
Take into account puberty and change into adult life. It´s important to screen comorbidity and detect it early, to optimize their development, treat comorbidity and prevent situations of emergency. 
Graphic of the revisions (medical).
Provides recommendations for certain problems, like motricity, respiratory disorders, etc.
Breathing problems don´t have treatment (except sleep apnea). The aim of general practitionnaire is to detect these sleep apneas by poli and asses nutritional impact. 
Cardiac problems: 15% of rett girls have prolongation of QTc and reduced heart pace. Avoid medications (fluoxetine is mentioned dire

### Step 6

Repeat the previous step for the full document and save the document as output.

In [13]:
output_result = []

for session in sessions:
    input_str = session["content"]

    print(f"Processing session: {session['title']}")

    # Replace abbreviations and acronyms from the input text
    abbreviations_expansion = asyncio.create_task(kernel.run_async(my_functions["AbbreviationExpansion"], input_str=input_str))
    abbreviations_expansion_result = await abbreviations_expansion

    # Review notes from the previous step, and summarize the notes
    note_review = asyncio.create_task(kernel.run_async(my_functions["NoteReview"], input_str=abbreviations_expansion_result.result))
    note_review_result = await note_review

    # Create the final session report, after the previous steps
    report = asyncio.create_task(kernel.run_async(my_functions["Summarize"], input_str=abbreviations_expansion_result.result))
    report_result = await report

    # Translate to Spanish
    context = ContextVariables()
    context["language"] = "spanish"
    context["input"] = report_result.result
    translate = asyncio.create_task(kernel.run_async(my_functions["TranslateNotes"], input_vars=context))
    translate_result = await translate

    output_result.append({"title": session["title"], "content": translate_result.result})

# Save the output to a Word document
output_document = Document()

for session in output_result:
    output_document.add_heading(session["title"], level=1)
    output_document.add_paragraph(session["content"])

output_document.save("output.docx")

Processing session: Health data applied to rare diseases: review of the national plan for rare diseases 3 and prospects for the national plan for rare diseases 4 in 2024. Anne-Sophie LAPOINTE (France)
Processing session: PNDS or Rett center. Nadia BAHI-BUISSON (France)
